In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings

sns.set()
rcParams['figure.figsize'] = (20,10)
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from pprint import pprint
import json

from transformers import AutoTokenizer, AutoModel, AutoConfig, pipeline
from datasets import load_dataset, Dataset, concatenate_datasets
import swifter

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

In [36]:
temp = Dataset.from_json('./temp.jsonl')
temp = temp.filter(lambda x: x['pred'] != '')

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-324e22197086a422/0.0.0)


Filter:   0%|          | 0/184 [00:00<?, ? examples/s]

In [97]:
a = temp.shuffle()[0]
print(a['question'])
print(a['answer'])
print(a['pred'])

Ai là người có quyền lực cao nhất Paris thời gian sau khi bỏ chế độ phong kiến?
['nhà thiên văn Jean Sylvain Bailly', 'Quốc hội', 'Quốc hội', 'Quốc hội']
Thị trưởng Jacques de Flesselles
Nhà thiên văn Jean Sylvain Bailly
Thị trưởng đầu tiên của Paris
Tuileries
Quốc hội lập hiến
Tuileries


### CNN_DAILY

In [ ]:
cnn = Dataset.from_json('../../data/translated/cnn_dailymail_30k_samples_len_100_1200_words_translated.json')

In [ ]:
input_template="Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] Tóm tắt ngắn gọn đoạn văn bản sau đây:\n{article_translated}\n[|AI|] "
def mapper(x):
    return {
        'input': input_template.format(article_translated=x['article_translated']),
        'output': 'Đoạn văn bản đã được tóm tắt:\n' + x['highlights_translated'] + '\n[|Con người|]'
    }

temp_cnn = cnn.shuffle().select(range(15000)).map(mapper,batched=False, remove_columns=['article_translated','highlights_translated'])

In [ ]:
print(temp_cnn[1]['input'] + temp_cnn[1]['output'])

In [ ]:
length = temp_cnn.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['input'] + x['output']))})
length.to_pandas().hist(bins=100)

In [ ]:
temp_cnn[0]

In [ ]:
temp_cnn.to_json('../../data/training_3_8/cnn_15k_samples.jsonl', orient='records', lines=True)

### DIALOG SUMMARY

In [ ]:
dialog = Dataset.from_json('../../data/translated/dialogsum_10k_samples_len_50_words_translated.json')

In [ ]:
dialog

In [ ]:
input_template="Tóm tắt ngắn gọn đoạn hội thoại sau đây:\n{dialogue_translated}\nĐoạn hội thoại đã được tóm tắt:\n"
def mapper(x):
    return {
        'input': input_template.format(dialogue_translated=x['dialogue_translated']),
        'output': x['summary_translated']
    }

temp_dialog = dialog.shuffle().select(range(10000)).map(mapper,batched=False, remove_columns=['dialogue_translated','summary_translated'])

In [ ]:
print(temp_dialog[1]['input'] + temp_dialog[1]['output'])

In [ ]:
temp_dialog[0]

In [ ]:
temp_dialog.to_json('../../data/training_3_8/dialogsum_10k_samples.jsonl', orient='records', lines=True)

### FAQS

In [ ]:
from fuzzywuzzy import fuzz
faqs = Dataset.from_json('../../../data/training_31_7/faq_chat_10k.jsonl')
print(faqs.shuffle()[0]['input'])

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
text2 = ' Nếu không đủ thông tin để trả lời thì trả lời: Tôi không biết.'
text3 = 'Điều luật liên quan: '
text4 = ' Chỉ được trả lời dựa trên điều luật được cung cấp.'
def mapper(x):
    return {
        'input': text,
        'output': x['input'].removeprefix(text).replace(text2,'').replace(text3, 'Điều luật liên quan:\n').replace(text4, '') + '\n[|Con người|]',
    }

def mapper2(x):
    ai_index = x['output'].find('[|AI|]')
    return {
        'input': x['input'] + x['output'][:ai_index + 7],
        'output': x['output'][ai_index + 7:],
    }

temp_faqs = faqs.map(mapper, batched=False).map(mapper2, batched=False)

In [ ]:
temp_faqs_df = temp_faqs.to_pandas()
temp_faqs_df = temp_faqs_df.drop_duplicates(subset=['output']).reset_index(drop=True)
temp_faqs_df['length_input'] = temp_faqs_df['input'].apply(lambda x: len(tokenizer.tokenize(x)))
temp_faqs_df['length_output'] = temp_faqs_df['output'].apply(lambda x: len(tokenizer.tokenize(x)))
temp_faqs_df['similarity'] = temp_faqs_df.apply(lambda x: fuzz.ratio(x['input'], x['output']) / (x['length_input'] + x['length_output']), axis=1)
temp_faqs_df

In [ ]:
temp_faqs_df['length_input'].hist(bins=50)

In [ ]:
temp_faqs_df['length_output'].hist(bins=50)

In [ ]:
temp_faqs_df['similarity'].hist(bins=50)

In [ ]:
temp_1 = temp_faqs_df[(temp_faqs_df['length_output'] <= 0.6 * temp_faqs_df['length_input']) & (temp_faqs_df['length_output'] <= 300) & (temp_faqs_df['similarity'] <= 0.07)].reset_index(drop=True)
temp_1

In [ ]:
temp_faqs = Dataset.from_pandas(temp_1)
temp = temp_faqs.shuffle()[0]
print(temp['input'])
print(temp['output'])

In [ ]:
from random import choice

text = 'Trả lời câu hỏi pháp luật dựa vào những điều luật liên quan dưới đây.'
instruction_templates = {
    0: text,
    1: 'Giúp tôi trả lời câu hỏi pháp luật dựa vào những điều luật dưới đây.',
    2: 'Trả lời câu hỏi pháp luật sau đây.',
    3: 'Dựa vào những điều luật được cung cấp, hãy trả lời câu hỏi pháp luật sau đây.',
    4: 'Sau đây là những điều luật liên quan đến câu hỏi pháp luật. Hãy trả lời câu hỏi pháp luật sau đây.',
}

question_templates = {
    0: 'Câu hỏi pháp luật: ',
    1: 'Câu hỏi: ',
    2: '',
}

cite_templates = {
    0: 'Điều luật liên quan:',
    1: 'Điều luật:',
}
def mapper(x):
    a = choice([0,0,0,1,2,3,4])
    b = choice([0,0,0,1,1,1,2])
    c = choice([0,1])
    return {
        'input': x['input'].replace(text, instruction_templates[a]).replace('Câu hỏi: ', question_templates[b]).replace('Điều luật liên quan:', cite_templates[c]),
        'output': x['output']
    }

temp_faqs = temp_faqs.map(mapper, batched=False)

In [ ]:
temp = temp_faqs.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
temp_faqs

In [ ]:
temp_faqs.remove_columns(['length_input', 'length_output'])
temp_faqs.to_json('../../../data/training_21_8/faqs_400_samples.jsonl', orient='records', lines=True)

### SHARE_GPT

In [ ]:
share = pd.read_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/share_gpt_no_code_conversations_40k.json', orient='records', lines=True)
share.columns = ['input']
share.dropna(inplace=True)
share.reset_index(drop=True, inplace=True)
share = Dataset.from_pandas(share)

In [ ]:
share = share.filter(lambda x: len(tokenizer.tokenize(x["input"])) < 2000)

In [ ]:
share.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['input']))}).to_pandas().hist(bins=100)

In [ ]:
# text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
text = 'The conversation between human and AI assistant.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['input'].removeprefix(text) + '\n[|Con người|]'
    }
share = share.map(mapper, batched=False)

In [ ]:
share

In [ ]:
share[0]

In [ ]:
share

In [ ]:
share.to_json('../../data/training_english/share_gpt_38k.jsonl', orient='records', lines=True)

### QUORA

In [ ]:
quora = Dataset.from_json('../../../data/translated/quora_chat_data_translated.json')

In [ ]:
print(quora.shuffle()[0]['prompt'])

In [ ]:
quora.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['prompt']))}).to_pandas().hist(bins=100)

In [ ]:
temp_quora = quora.shuffle().select(range(20000))
temp_quora

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['prompt'].removeprefix(text)
    }

temp_quora = temp_quora.map(mapper, batched=False, remove_columns=['prompt'])

In [ ]:
temp = temp_quora.shuffle()[0]
print(temp['input'])
print(temp['output'])

In [ ]:
temp_quora.shuffle().select(range(5000)).to_json('../../../data/training_21_8/quora_5k_samples.jsonl', orient='records', lines=True)

### Oasst

In [ ]:
oasst = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/translated/oasst_translated.json')
oasst = oasst.filter(lambda x: not ('```' in x['text'] or 'trăn' in x['text'] or "'''" in x['text']))
oasst

In [ ]:
temp_oasst2 = oasst.filter(lambda x: len(tokenizer.tokenize(x['text'])) > 100)

In [ ]:
temp_oasst2

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['text'].removeprefix(text)
    }

temp_oasst = temp_oasst2.map(mapper, batched=False, remove_columns=['text'])

In [ ]:
temp = temp_oasst.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
temp_oasst.shuffle().select(range(5000)).to_json('../../../data/training_21_8/oasst_5k_samples.jsonl', orient='records', lines=True)

In [ ]:
oasst.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['text']))}).to_pandas().hist(bins=100)

In [ ]:
print(temp_oasst.shuffle()[0])

In [ ]:
temp_oasst.to_json('../../data/training_3_8/oasst_20k_samples.jsonl', orient='records', lines=True)

### ALPACA CHAT

In [ ]:
alpaca = Dataset.from_json('../../../data/translated/alpaca_chat_cleaned_51k_translated.json')
temp_df = alpaca.to_pandas()
temp_length = temp_df.apply(lambda x: len(tokenizer.tokenize(x['prompt'])), axis=1)

In [ ]:
alpaca_temp = Dataset.from_pandas(temp_df[temp_length>=150].reset_index(drop=True))
alpaca_temp

In [ ]:
print(alpaca_temp.shuffle()[0]['prompt'])

In [ ]:
def mapper(x):
    ai_index = x['prompt'].find('[|AI|]')
    return {
        'input': x['prompt'][:ai_index+7],
        'output': x['prompt'][ai_index + 7:]
    }
temp_alpaca = alpaca_temp.map(mapper, batched=False, remove_columns=alpaca.column_names)

In [ ]:
temp_alpaca.shuffle()[0]

In [ ]:
temp_alpaca.to_json('../../../data/training_14_8/alpaca_chat_26k_samples.jsonl', orient='records', lines=True)

### GPT4 INSTRUCT

In [ ]:
gpt4 = Dataset.from_json('../../../data/translated/gpt4_instruct_similarity_0_9_translated.jsonl')
gpt4

In [ ]:
template1 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] {instruction_translated}\n{input_translated}\n[|AI|] "
template2 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] {instruction_translated}\n[|AI|] "

def mapper(x):
    if x['input_translated'] != '' and x['input_translated'] is not None:
        template = template1.format(
            instruction_translated=x['instruction_translated'], 
            input_translated=x['input_translated']
        )
    else:
        template = template2.format(instruction_translated=x['instruction_translated'])
    return {
        'input': template,
        'output': x['response_translated'] + '\n[|Con người|]'
    }

temp_gpt4 = gpt4.map(mapper,batched=False, remove_columns=gpt4.column_names)

In [ ]:
temp = temp_gpt4.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
a = temp_gpt4.shuffle()[0]
print(a['input']+a['output'])

In [ ]:
temp_gpt4.to_json('../../data/training_3_8/gpt4_instruct.jsonl', orient='records', lines=True)

### DOLLY WITH CONTEXT

In [ ]:
dolly = Dataset.from_json('../../../data/translated/dolly_with_context_translated.jsonl')
# dolly = dolly.remove_columns(['instruction', 'context', 'response', 'category_translated'])

In [ ]:
temp = dolly.shuffle()[0]
print(temp['context_translated'])
print('----------')
print(temp['instruction_translated'])

In [ ]:
dolly.shuffle()[0]

In [ ]:
from random import choice

pretext = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] '
instruction_templates = [
    "Trả lời câu hỏi dựa vào đoạn văn bản dưới đây. Nếu không có thông tin trong đoạn văn bản thì trả lời: Tôi không biết.\nCâu hỏi: {instruction_translated}\nĐoạn văn bản:\n{context_translated}\n[|AI|] ",
    "Giúp tôi trả lời câu hỏi sau đây.\nCâu hỏi: {instruction_translated}\nĐoạn văn bản:\n{context_translated}\n[|AI|] ",
    "Sử dụng thông tin trong văn bản sau đây để trả lời câu hỏi.\nVăn bản:\n{context_translated}\nCâu hỏi: {instruction_translated}\n[|AI|] ",
    "Hãy trả lời câu hỏi sau sử dụng thông tin của đoạn văn bản được cung cấp, nếu không có thông tin thì phản hồi không trả lời được.\n{context_translated}\nCâu hỏi: {instruction_translated}\n[|AI|] ",
    "Trả lời câu hỏi sử dụng đoạn văn bản sau.\n{context_translated}\n{instruction_translated}\n[|AI|] ",
    "Đưa ra phản hồi cho câu hỏi sau đây, nếu không tìm thấy thông tin thì trả lời: Không có thông tin cho câu hỏi của bạn.\n{context_translated}\nCâu hỏi: {instruction_translated}\n[|AI|] ",
]

def mapper(x):
    ip = pretext + choice(instruction_templates).format(
        instruction_translated=x['instruction_translated'],
        context_translated=x['context_translated']
    )
    return {
        'input': ip,
        'output': choice([''] * 9 + ['Trả lời: ', 'Câu trả lời: ']) + x['response_translated'] + '\n[|Con người|]'
    }

temp_dolly = dolly.map(mapper,batched=False, remove_columns=dolly.column_names)

In [ ]:
temp = temp_dolly.shuffle()[0]
print(temp['input'] + temp['output'])

Cuộc trò chuyện giữa con người và trợ lý AI.
[|Con người|] Trả lời câu hỏi pháp luật dựa vào những điều luật liên quan dưới đây. Chỉ được trả lời dựa trên thông tin nằm trong điều luật được cung cấp.
Câu hỏi: Phạm tội trong trạng thái tinh thần bị kích động mạnh bị xử lý thế nào?
Điều luật liên quan:
Điều 125. Tội giết người trong trạng thái tinh thần bị kích động mạnh 1. Người nào giết người trong trạng thái tinh thần bị kích động mạnh do hành vi trái pháp luật nghiêm trọng của nạn nhân đối với người đó hoặc đối với người thân thích của người đó, thì bị phạt tù từ 06 tháng đến 03 năm. 2. Phạm tội đối với 02 người trở lên, thì bị phạt tù từ 03 năm đến 07 năm.
[|AI|] Giết người trong trạng thái tinh thần bị kích động mạnh Tội giết người trong trạng thái tinh thần bị kích động mạnh được quy định tại Điều 125 Bộ luật Hình sự 2015 sửa đổi, bổ sung 2017 với mức hình phạt như sau: Khung 1: Người nào giết người trong trạng thái tinh thần bị kích động mạnh do hành vi trái pháp luật nghiêm trọng của nạn nhân đối với người đó hoặc đối với người thân thích của người đó, thì bị phạt tù từ 06 tháng đến 03 năm. Khung 2: Phạm tội đối với 02 người trở lên, thì bị phạt tù từ 03 năm đến 07 năm.
[|Con người|]

In [ ]:
Dataset.from_pandas(temp_dolly.to_pandas().sample(5000, replace=True).reset_index(drop=True)).to_json('../../../data/training_21_8/dolly_4k_samples.jsonl', orient='records', lines=True)

### Role Play

In [ ]:
role = Dataset.from_json('../../../data/translated/roleplay_translated.jsonl')
role = Dataset.from_pandas(role.data.to_pandas().sample(5000, replace=True))

In [ ]:
template1 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] {instruction_translated}\n[|AI|] "
template2 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] {instruction_translated}\n{input_translated}\n[|AI|] "

def mapper(x):
    if x['input_translated'] == '':
        ip = template1.format(
            instruction_translated=x['instruction_translated'], 
        )
    else:
        ip = template2.format(
            instruction_translated=x['instruction_translated'], 
            input_translated=x['input_translated']
        )
    return {
        'input': ip,
        'output': x['output_translated'] + '\n[|Con người|]'
    }

temp_role = role.map(mapper,batched=False, remove_columns=role.column_names)

In [ ]:
temp_role

In [ ]:
temp = temp_role.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
temp_role.to_json('../../../data/training_14_8/role_play_3k_samples.jsonl', orient='records', lines=True)

### Orca

In [ ]:
orca = Dataset.from_json('../../../data/translated/orca_translated_50k.jsonl')
orca = orca.remove_columns(['system_prompt','question','response'])

In [ ]:
temp = orca.shuffle()[0]
print(temp['system_prompt_translated'])
print('------------------')
print(temp['question_translated'])
print('------------------')
print(temp['response_translated'])

In [ ]:
template1 = "{system_prompt}\n[|Con người|] {question}\n[|AI|] "
template2 = "[|Con người|] {question}\n[|AI|] "

def mapper(x):
    if x['system_prompt_translated'] == '':
        ip = template2.format(
            question=x['question_translated'],
        )
    else:
        ip = template1.format(
            system_prompt=x['system_prompt_translated'],
            question=x['question_translated'],
        )
    return {
        'input': ip,
        'output': x['response_translated'] + '\n[|Con người|]',
    }

temp_orca = orca.map(mapper, remove_columns=orca.column_names)


In [ ]:
temp = temp_orca.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
temp_orca.shuffle().select(range(25000)).to_json('../../../data/training_21_8/orca_25k_samples.jsonl', orient='records', lines=True)

### Wivi

In [ ]:
wivi = Dataset.from_json('../../../data/translated/wizard_vicuna_nocode_translated.jsonl')
wivi = wivi.select_columns(['conversations_translated'])
wivi = wivi.rename_column('conversations_translated', 'conversation')

In [ ]:
wivi[0]['conversation']

In [ ]:
text = "Cuộc trò chuyện giữa con người và trợ lý AI.\n"
def mapper(x):
    return {
        'input': text,
        'output': x['conversation'].removeprefix(text) + '\n[|Con người|]'
    }

temp_wivi = wivi.map(mapper, batched=False, remove_columns=wivi.column_names)

In [ ]:
temp = temp_wivi.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
temp_wivi.to_json('../../../data/training_21_8/wivi_30k_samples.jsonl', orient='records', lines=True)

### Conversation query generation

In [4]:
data_path = [
    "../../../data/translated/retrieve_query_gen_oasst_translated.jsonl", 
    "../../../data/translated/retrieve_query_gen_quora_translated.jsonl", 
    "../../../data/translated/retrieve_query_gen_sharegpt_translated.jsonl",
]
query = concatenate_datasets([Dataset.from_json(path) for path in data_path])
temp = query.to_pandas().drop_duplicates(subset=['conversation']).sample(500, replace=True)
query = Dataset.from_pandas(temp.reset_index(drop=True))

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-3ecfe4735495290c/0.0.0)


Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-0bad7c81935852e3/0.0.0)
Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-63a04f4664c48905/0.0.0)


In [5]:
query

Dataset({
    features: ['conversation', 'conversation_translated', 'query', 'query_translated'],
    num_rows: 500
})

In [6]:
str("Hello from hoang").find("hoang")

11

In [7]:
template1 = "Dưới đây là đoạn hội thoại giữa con người và AI, cuối đoạn hội thoại là câu hỏi mới người dùng đặt ra cần được trả lời bằng cách tìm kiếm thông tin nằm trong cơ sở tri thức chung. Hãy sinh ra câu truy vấn phù hợp với đoạn hội thoại và câu hỏi mới của người dùng.\nĐoạn hội thoại:\n{conversation}\nCâu truy vấn: "

def mapper(x):
    ip = template1.format(
        conversation=x['conversation_translated'], 
    )
    return {
        'input': ip,
        'output': x['query_translated']
    }

temp_query = query.map(mapper,batched=False, remove_columns=query.column_names)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
temp = temp_query.shuffle()[0]
print(temp['input'] + temp['output'])

Dưới đây là đoạn hội thoại giữa con người và AI, cuối đoạn hội thoại là câu hỏi mới người dùng đặt ra cần được trả lời bằng cách tìm kiếm thông tin nằm trong cơ sở tri thức chung. Hãy sinh ra câu truy vấn phù hợp với đoạn hội thoại và câu hỏi mới của người dùng.
Đoạn hội thoại:
[|Con người|] Hãy giải thích cho tôi giống như tôi 7 tuổi, trí tuệ nhân tạo là gì.
[|AI|] Chắc chắn rồi, tôi có thể giải thích cho bạn theo cách mà bạn có thể hiểu được!

Trí tuệ nhân tạo (AI) là một chương trình máy tính hoặc máy móc có thể làm những việc mà thông thường đòi hỏi trí thông minh của con người phải làm. Nó có thể học hỏi kinh nghiệm và cải thiện theo thời gian, giống như cách bạn trở nên giỏi hơn trong một trò chơi khi bạn chơi nó nhiều hơn.

Ví dụ, bạn đã bao giờ chơi một trò chơi mà máy tính là đối thủ của bạn chưa? Máy tính đưa ra quyết định và chơi trò chơi giống như người thật, nhưng thực tế nó chỉ là một chương trình được thiết kế để làm điều đó. Đó là một ví dụ về AI!

AI được sử dụng trong

In [ ]:
temp_query.to_json('../../../data/training_14_8/query_gen_500_samples.jsonl', orient='records', lines=True)

### Wizard

In [ ]:
wizard = Dataset.from_json('../../../data/translated/wizard_no_code_translated.jsonl')
wizard

In [ ]:
temp = wizard.shuffle()[0]
print(temp['conversations_translated'])

In [ ]:
text = "Cuộc trò chuyện giữa con người và trợ lý AI.\n"
def mapper(x):
    index = x['conversations_translated'].find('[|AI|]')
    return {
        'input': text + x['conversations_translated'][:index+7],
        'output': x['conversations_translated'][index+7:] + '\n[|Con người|]'
    }

temp_wizard = wizard.map(mapper)

In [ ]:
temp = temp_wizard.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
temp_wizard.shuffle().select(range(40000)).to_json('../../../data/training_21_8/wizard_30k_samples.jsonl', orient='records', lines=True)

### Okapi

In [ ]:
code_ocapi = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/okapi/code_related_okapi.jsonl')
no_code_ocapi = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/okapi/okapi_instruct_vi.json')

In [ ]:
template1 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] {instruction}\n{input}\n[|AI|] "
template2 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] {instruction}\n[|AI|] "

def mapper(x):
    if x['input'] == '':
        ip = template2.format(
            instruction=x['instruction'],
        )
    else:
        ip = template1.format(
            instruction=x['instruction'],
            input=x['input']
        )
    return {
        'input': ip,
        'output': x['output'] + '\n[|Con người|]'
    }

temp_code_ocapi = code_ocapi.map(mapper, remove_columns=code_ocapi.column_names)    
temp_no_code_ocapi = no_code_ocapi.map(mapper, remove_columns=no_code_ocapi.column_names)

In [ ]:
temp_code_ocapi.to_json('../../../data/training_21_8/code_related_ocapi_6k_samples.jsonl', orient='records', lines=True)
temp_no_code_ocapi.shuffle().select(range(30000)).to_json('../../../data/training_21_8/no_code_ocapi_30k_samples.jsonl', orient='records', lines=True)

In [ ]:
temp = temp_no_code_ocapi.shuffle()[0]
print(temp['input'] + temp['output'])

### MMLU

In [ ]:
mmlu_test = Dataset.from_json('../../../data/translated/mmlu/zero_shot_mmlu_chat_test_translated.jsonl')
mmlu_val = Dataset.from_json('../../../data/translated/mmlu/zero_shot_mmlu_chat_val_translated.jsonl')
print(len(mmlu_test), len(mmlu_val))
def mapper(x):
    return {
        'input': x['input_translated'] + "\n[|AI|] ",
        'output': x['output_translated'].removeprefix("[|AI|] ").replace("[|Human|]", "[|Con người|]")
    }
def filter(x):
    for a in ['A.', 'B.', 'C.', 'D.']:
        if not a in x['input']:
            return False
    for a in ['A\n[|Con người|]', 'B\n[|Con người|]', 'C\n[|Con người|]', 'D\n[|Con người|]']:
        if a in x['output']:
            return True
mmlu_val = mmlu_val.map(mapper).filter(filter)
mmlu_test = mmlu_test.map(mapper).filter(filter)
print(len(mmlu_test), len(mmlu_val))

In [ ]:
temp = mmlu_test.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
mmlu_val.to_json('../../../data/training_21_8/eval_test/mmlu_zeroshot_val.jsonl', orient='records', lines=True)
mmlu_test.to_json('../../../data/training_21_8/eval_test/mmlu_zeroshot_test.jsonl', orient='records', lines=True)

### Squad V2 (No answer)

In [ ]:
from random import choice
squadv2 = Dataset.from_json('../../../data/translated/squad2_no_ans_translated.jsonl').shuffle().select(range(1500))


pre_text = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] "

prompts1 = [
    'Hãy trả lời câu hỏi sau\n{question}\nThông tin:\n{context}',
    'Phản hồi lại câu hỏi sau đây sử dụng thông tin trong đoạn văn bản.\nVăn bản:\n{context}\nCâu hỏi: {question}',
    "{context}\nCâu hỏi: {question}",
    'Trả lời câu hỏi dựa vào đoạn văn bản sau đây.\nĐoạn văn bản:\n{context}\nCâu hỏi:\n{question}',
    'Trả lời câu hỏi sau đây, chỉ sử dụng thông tin nằm trong đoạn văn bản.\nĐoạn văn bản:\n{context}\n{question}',
    'Sử dụng thông tin trong đoạn trích để trả lời câu hỏi sau:\n{question}\nĐoạn trích:\n{context}',
]

prompts2 = [
    ["Trả lời câu hỏi dựa vào thông tin trong đoạn văn bản sau đây. Nếu không tìm thấy thống tin nằm trong đoạn văn bản thì trả lời: Không có thông tin cho câu hỏi của bạn.\n{context}\nCâu hỏi: {question}\n[|AI|] ", "Không có thông tin cho câu hỏi của bạn.\n[|Con người|]"],
    ["Sử dụng thông tin trong đoạn văn bản sau đây để trả lời câu hỏi. Nếu không tìm thấy thống tin nằm trong đoạn văn bản thì trả lời: Không có thông tin cho câu hỏi của bạn.\nCâu hỏi: {question}\nĐoạn văn bản:\n{context}\n[|AI|] ", "Không có thông tin cho câu hỏi của bạn.\n[|Con người|]"],
    ["Sử dụng đoạn trích sau đây để trả lời, nếu không có thông tin thì phản hồi không trả lời được.\n{context}\nCâu hỏi: {question}\n[|AI|] ","Xin lỗi, tôi không có câu trả lời cho câu hỏi của bạn.\n[|Con người|]"],
    ["Hãy trả lời câu hỏi sau đây, nếu không có thông tin thì trả lời không trả lời được.\nVăn bản:\n{context}\n{question}\n[|AI|] ","Xin lỗi, tôi không trả lời được câu hỏi này.\n[|Con người|]"],
]
    
no_answers = [
    "Xin lỗi, tôi không có thông tin cho câu hỏi của bạn.\n[|Con người|]",
    "Tôi không có câu trả lời cho câu hỏi này.\n[|Con người|]",
    "Xin lỗi bạn, tôi không có câu trả lời.\n[|Con người|]",
    "Không có câu trả lời cho câu hỏi của bạn.\n[|Con người|]",
    "Xin lỗi, tôi không thể trả lời câu hỏi này.\n[|Con người|]",
]
def mapper1(x):
    return {
        'input': (pre_text + choice(prompts1) + "\n[|AI|] ").format(question=x['question_translated'], context=x['context_translated']),
        'output': choice(no_answers)
    }

def mapper2(x):
    c = choice(prompts2)
    return {
        'input': (pre_text + c[0]).format(question=x['question_translated'], context=x['context_translated']),
        'output': c[1]
    }


temp = squadv2.train_test_split(test_size=0.5, seed=42)
final_1 = temp['train'].map(mapper1, remove_columns=squadv2.column_names)
final_2 = temp['test'].map(mapper2, remove_columns=squadv2.column_names)
print(final_1, final_2)

In [ ]:
temp = final_1.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
temp = final_2.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
final = concatenate_datasets([final_1, final_2])
final.to_json('../../../data/training_21_8/squadv2_no_ans_2000_samples.jsonl', orient='records', lines=True)

In [ ]:
temp = final.shuffle()[0]
print(temp['input'] + temp['output'])

### Viquad

In [26]:
visquad_1 = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/translated/vi_squad/dev.json')
visquad_2 = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/translated/vi_squad/test.json')

def extract_context_question_answer(ds):
    results = []
    for i in ds[0]['data']:
        for j in i['paragraphs']:
            for k in j['qas']:
                results.append({
                    'context': j['context'],
                    'question': k['question'],
                    'answer': [a['text'] for a in k['answers']]
                })
    return Dataset.from_pandas(pd.DataFrame(results))

temp1 = extract_context_question_answer(visquad_1)
temp2 = extract_context_question_answer(visquad_2)
visquad = concatenate_datasets([temp1, temp2])

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-d0864ec1de14c06f/0.0.0)
Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-75e4a36a86b0b199/0.0.0)


In [34]:
visquad.to_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/translated/vi_squad/vi_squad_test.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

9895384

In [22]:
visquad_1[0]['data'][0]['paragraphs'][0]['qas'][0]['answers']

[{'answer_start': 6,
  'text': 'nằm ở điểm gặp nhau của các hành trình thương mại đường bộ và đường sông'},
 {'answer_start': 12,
  'text': 'điểm gặp nhau của các hành trình thương mại đường bộ và đường sông, và là trung tâm của một vùng nông nghiệp giàu có'},
 {'answer_start': 12,
  'text': 'điểm gặp nhau của các hành trình thương mại đường bộ và đường sông, và là trung tâm của một vùng nông nghiệp giàu có'},
 {'answer_start': 12,
  'text': 'điểm gặp nhau của các hành trình thương mại đường bộ và đường sông, và là trung tâm của một vùng nông nghiệp giàu có'}]

In [ ]:
visquad_1[0]

In [ ]:
visquad[0]

### Query topic

In [ ]:
topic_names = ["văn hoá xã hội", "toán và khoa học", "sức khoẻ", "tư vấn và giáo dục", "máy tính và internet", "thể thao", "kinh doanh và kinh tế", "âm nhạc và giải trí", "gia đình và các mối quan hệ", "pháp luật"]

topic_map = {
    str(k): v for k, v in zip(range(len(topic_names)), topic_names)
}
topic = Dataset.from_json('../../../data/translated/query_topic_translated.jsonl')
topic_df = topic.to_pandas()
topic_df = topic_df[topic_df['topic'].apply(lambda x: x != '9')]
topic_df['topic_name'] = topic_df['topic'].apply(lambda x: topic_map[x])
topic = Dataset.from_pandas(topic_df).select_columns(['topic_name', 'question_translated']).rename_column('question_translated', 'question').shuffle().select(range(3250))

faqs = Dataset.from_json('../../../data/translated/all_faqs.json')
def mapper(x):
    i = x['input'].find('\nĐiều luật liên quan: ')
    return {
        'question': x['input'][:i].removeprefix('Câu hỏi: '),
        'topic_name': 'pháp luật'
    }
faqs = faqs.map(mapper, remove_columns=faqs.column_names)
faqs = faqs.shuffle().select(range(750))

topic = concatenate_datasets([topic, faqs]).shuffle()

In [ ]:
topic.to_pandas()['topic_name'].value_counts().plot(kind='bar')

In [ ]:
topic

In [ ]:
from random import choice
pretext = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] "
type1_input_prompts = [
    "Cho biết câu hỏi sau thuộc chủ đề nào?\n{question}\n[|AI|] ",
    "Câu hỏi sau đây thuộc chủ đề nào?\n{question}\n[|AI|] ",
    "Sau đây là một câu hỏi, hãy cho biết nó thuộc chủ đề nào?\nCâu hỏi: {question}\n[|AI|] ",
    "\nHỏi: {question}\nCâu hỏi trên thuộc chủ đề nào?\n[|AI|] ",
    "\n{question}\nChủ đề của câu hỏi trên là gì?\n[|AI|] ",
    "Giúp tôi xác định chủ đề của câu hỏi sau:\n{question}\n[|AI|] ",
]
type1_output_prompts = [
    "Câu hỏi này thuộc chủ đề {topic_name}.\n[|Con người|]",
    "{topic_name}.\n[|Con người|]",
    "Chủ đề của câu hỏi này là {topic_name}.\n[|Con người|]",
]    

def type1_prompt(x):
    return {
        'input': (pretext + choice(type1_input_prompts)).format(question=x['question']),
        'output': choice(type1_output_prompts).format(topic_name=x['topic_name'])
    }

type2_input_prompts = [
    "Câu hỏi sau đây có thuộc chủ đề {topic_name} không?\nCâu hỏi: {question}\n[|AI|] ",
    "Đây có phải là câu hỏi thuộc chủ đề {topic_name} không?\n{question}\n[|AI|] ",
    "Cho biết nếu câu hỏi sau đây thuộc chủ đề {topic_name}?\n{question}\n[|AI|] ",
]
type2_output_prompts = [
    "Đúng vậy, câu hỏi này thuộc chủ đề {topic_name}.\n[|Con người|]",
    "Chính xác.\n[|Con người|]",
    "Đúng vậy.\n[|Con người|]",
]

def type2_prompt(x):
    return {
        'input': (pretext + choice(type2_input_prompts)).format(question=x['question'], topic_name=x['topic_name']),
        'output': choice(type2_output_prompts).format(topic_name=x['topic_name'])
    }

type3_input_prompts = [
    "Câu hỏi sau đây có thuộc chủ đề {other_topic_name} không?\nHỏi: {question}\n[|AI|] ",
    "Đây có phải là câu hỏi thuộc chủ đề {other_topic_name} không?\n{question}\n[|AI|] ",
    "Câu hỏi này có liên quan đến chủ đề {other_topic_name} không?\nCâu hỏi: {question}\n[|AI|] ",
    "Cho biết nếu câu hỏi sau đây thuộc chủ đề {other_topic_name}?\n{question}\n[|AI|] ",
    "{question}\nCâu hỏi trên có liên quan đến chủ đề {other_topic_name} không?\n[|AI|] ",
]
type3_output_prompts = [
    "Không, câu hỏi này không thuộc chủ đề {other_topic_name}.\n[|Con người|]",
    "Không đúng.\n[|Con người|]",
    "Không đúng, câu hỏi này thuộc chủ đề {topic_name}.\n[|Con người|]",
    "Chủ đề của câu hỏi này là {topic_name} chứ không phải {other_topic_name}.\n[|Con người|]",
]

def type3_prompt(x):
    other = choice([topic for topic in topic_names if topic != x['topic_name']])
    return {
        'input': (pretext + choice(type3_input_prompts)).format(question=x['question'], other_topic_name=other),
        'output': choice(type3_output_prompts).format(topic_name=x['topic_name'], other_topic_name=other)        
    }

temp = topic.train_test_split(test_size=2400, seed=42)
type3_data = temp['train']
temp = temp['test'].train_test_split(test_size=1200, seed=42)
type1_data = temp['train']
type2_data = temp['test']
print(len(type1_data), len(type2_data), len(type3_data))

final_type1 = type1_data.map(type1_prompt, remove_columns=topic.column_names)
final_type2 = type2_data.map(type2_prompt, remove_columns=topic.column_names)
final_type3 = type3_data.map(type3_prompt, remove_columns=topic.column_names)

topic_classification = concatenate_datasets([final_type1, final_type2, final_type3]).shuffle()

In [ ]:
a = topic_classification.shuffle()[0]
print(a['input'] + a['output'])

In [ ]:
topic_classification.to_json('../../../data/training_21_8/topic_classification_4000_samples.jsonl', orient='records', lines=True)

### Load all

In [ ]:
root_path = '/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/training_21_8/'
datasets = [
    ['dolly_5k_samples.jsonl', 4000], # 5000
    ['faqs_400_samples.jsonl', None], # 400
    ['squadv2_no_ans_1500_samples.jsonl', 600], # 500
    ['topic_classification_4000_samples.jsonl', 500], # 3000
    ['query_gen_500_up_samples.jsonl', None], # 500
    # ['code_related_ocapi_6k_samples.jsonl', None],
    # ['gpt4_instruct_10k.jsonl', None],
    # ['no_code_ocapi_30k_samples.jsonl', None],
    # ['oasst_5k_samples.jsonl', None],
    # ['orca_25k_samples.jsonl', None],
    # ['quora_5k_samples.jsonl', None],
    # ['wivi_30k_samples.jsonl', None],
    # ['wizard_40k_samples.jsonl', None],
]
def clean(ds, max_tokens=2048):
    def mapper(x):
        for column in ds.column_names:
            x[column] = x[column].replace('[|Human|]','[|Con người|]').replace('\n[|Con người|]\n[|Con người|]', '\n[|Con người|]').replace('\r\n', '\n')
        return x
    
    def len_mapper(x):
        input_len = len(tokenizer.tokenize(x['input']))
        output_len = len(tokenizer.tokenize(x['output']))
        return {
            'input_len': input_len,
            'output_len': output_len,
            'len': input_len + output_len
        }
    def filter(x):
        for column in ds.column_names:
            if x[column] == '' or x[column] is None:
                return False
        if x['len'] > max_tokens:
            return False
        return True

    return ds.map(mapper,batched=False).map(len_mapper, batched=False).filter(filter)

def load_datasets(dataset_size=None, max_tokens=2048):
    data = []
    for ds_config in datasets:
        ds = Dataset.from_json(root_path + ds_config[0])
        df = ds.to_pandas()
        df['source'] = ds_config[0].split('.')[0].split('_')[0]
        ds = Dataset.from_pandas(df)
        if ds_config[1] is not None:
            ds = ds.shuffle(seed=42).select(range(min(ds_config[1], len(ds))))
        for check in ['input', 'output']:
            if check not in ds.column_names:
                raise ValueError("Dataset must have input, output columns")
        # take only the input and output columns
        ds = ds.select_columns(['input', 'output'])
        print(ds_config[0], ds)
        data.append(ds)
    ds = concatenate_datasets(data, axis=0)
    if dataset_size is None: dataset_size = len(ds)
    ds = ds.shuffle(seed=42).select(range(dataset_size))
    return clean(ds, max_tokens=max_tokens)

In [ ]:
ds = load_datasets(max_tokens=1000)

In [ ]:
ds.to_pandas()['output_len'].quantile(0.2)

In [ ]:
other = ds.shuffle().select(range(54000))

In [ ]:
other_df = other.to_pandas()
other_df[other_df['output_len'] <= 50].sample(1)['output'].values[0]

In [ ]:
important = ds

In [ ]:
important

In [ ]:
final = concatenate_datasets([important, other]).shuffle()

In [ ]:
final.to_pandas()['output_len'].quantile(0.75)

In [ ]:
final.to_json('../../../data/training_21_8/60k_mixed_2.jsonl', orient='records', lines=True)

In [ ]:
final.to_pandas()['len'].hist(bins=100)

In [ ]:
temp = ds.filter(lambda x: x['len'] <= 1000)
t = temp.shuffle()[0]
print(t['input'] + t['output'])

In [ ]:
temp.to_pandas().hist(column='len', bins=100)

In [ ]:
temp.shuffle().select(range(60000)).to_json('../../../data/training_21_8/mix_60k.jsonl', orient='records', lines=True)

In [ ]:
temp = ds.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
df = ds.to_pandas()
df['len'].hist(bins=100)

In [ ]:
df[df['output_len'] <= 50]

In [ ]:
temp = df[df['output_len'] <= 40].sample(1)
print(temp['input'].values[0] + temp['output'].values[0])

In [ ]:
final_df = df[df['len'] <= 2000].reset_index(drop=True)
final_ds = Dataset.from_pandas(final_df).select_columns(['input', 'output'])
final_ds

In [ ]:
plt.figure(figsize=(20,10))
final_df['len'].hist(bins=100)

In [ ]:
final_ds

In [ ]:
final_ds.to_json('../../../data/training_21_8/all_21_8.jsonl', orient='records', lines=True)

In [ ]:
task_oriented = final_ds.shuffle()

In [ ]:
instructs = final_ds.shuffle().select(range(40000))

In [ ]:
final = concatenate_datasets([task_oriented, instructs]).shuffle()

In [ ]:
final.to_json('../../../data/training_21_8/final_51k_25_8_samples.jsonl', orient='records', lines=True)

In [ ]:
final

In [ ]:
final.to_pandas().hist(column='len', bins=100)

In [ ]:
final

In [ ]:
temp = final.shuffle()[0]
print(temp['input'] + temp['output'])

In [ ]:
ds = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/training_21_8/all_21_8.jsonl')

In [ ]:
def mapper(x):
    return {
        'input_len': len(tokenizer.encode(x['input'])),
        'output_len': len(tokenizer.encode(x['output']))
    }

ds = ds.map(mapper, batched=False)

In [ ]:
ds

In [ ]:
ds.to_pandas().hist(column='input_len', bins=50)
ds.to_pandas().hist(column='output_len', bins=50)

In [ ]:
ds.to_pandas()['output_len'].quantile(0.7)

### Length Distribution

In [ ]:
root_path = '/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/training_21_8/'
dataset_configs = [
    ['code_related_ocapi_6k_samples.jsonl', 'instruct'],
    ['dolly_5k_samples.jsonl', 'instruct'],
    ['faqs_400_samples.jsonl', 'instruct'],
    ['gpt4_instruct_10k.jsonl', 'instruct'],
    ['no_code_ocapi_30k_samples.jsonl', 'instruct'],
    ['query_gen_500_up_samples.jsonl', 'instruct'],
    ['orca_25k_samples.jsonl', 'instruct'],
    ['role_play_5k_up_samples.jsonl', 'instruct'],
    ['squadv2_no_ans_1500_samples.jsonl', 'instruct'],
    ['topic_classification_4000_samples.jsonl', 'instruct'],
    ['wizard_40k_samples.jsonl', 'instruct'],
    ['wivi_30k_samples.jsonl', 'chat'],
    ['quora_5k_samples.jsonl', 'chat'],
    ['oasst_5k_samples.jsonl', 'chat'],
]
def len_analysis(dataset_configs):
    def mapper(x):
        return {
            'input_len': len(tokenizer.encode(x['input'])),
            'output_len': len(tokenizer.encode(x['output']))
        }
    collections = {}
    for ds_config in dataset_configs:
        name = ds_config[0].split('.')[0]
        collections[name] = Dataset.from_json(root_path + ds_config[0])
    for k, v in collections.items():
        collections[k] = v.map(mapper, batched=False, remove_columns=v.column_names)
    for k, v in collections.items():
        print('Dataset: ', k)
        df = v.to_pandas()
        df['input_len'].hist(bins=50)
        plt.show()
        df['output_len'].hist(bins=50)
        plt.show()
        print('---------------------')

len_analysis(dataset_configs)

In [ ]:
DATASET = '/kaggle/input/general-chatbot-5-8/60k_mixed_2.jsonl'
DATASET_FORMAT = 'input-output'
OUTPUT_DIR = './outputs/'

PER_DEVICE_TRAIN_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = '0.03'
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03

LORA_R = 128
LORA_ALPHA = 32 
LORA_DROPOUT = 0.1

TRAIN_ON_SOURCE = False
SOURCE_MAX_LENGTH = 1024
TARGET_MAX_LENGTH = 1024

LOGGING_STEPS = 20
SAVE_STEPS = 100
SAVE_TOTAL_LIMIT = 4

def check(args):
    assert args.dataset == DATASET
    assert args.dataset_format == DATASET_FORMAT
    assert args.output_dir == OUTPUT_DIR
    assert args.per_device_train_batch_size == PER_DEVICE_TRAIN_BATCH_SIZE
    assert args.gradient_accumulation_steps == GRADIENT_ACCUMULATION_STEPS
    assert args.learning_rate == LEARNING_RATE
    assert args.lr_scheduler_type == LR_SCHEDULER_TYPE
    assert args.warmup_ratio == WARMUP_RATIO
    assert args.lora_r == LORA_R
    assert args.lora_alpha == LORA_ALPHA
    assert args.lora_dropout == LORA_DROPOUT
    assert args.train_on_source == TRAIN_ON_SOURCE
    assert args.source_max_length == SOURCE_MAX_LENGTH
    assert args.target_max_length == TARGET_MAX_LENGTH
    assert args.logging_steps == LOGGING_STEPS
    assert args.save_steps == SAVE_STEPS
    assert args.save_total_limit == SAVE_TOTAL_LIMIT
